### 과제

xml 정보로 custom dataset을 만들고,  이 스크립트가 돌아가면 됨

return 정보(이미지, 박스 정보, 카테고리 id 정보)를 넘겨서 받아가지고 함수에 넣어서 돌아가게끔

In [32]:
import random
import cv2
import albumentations as A
import json
from torch.utils.data.dataset import Dataset
import os
import glob
from PIL import Image


In [33]:
BOX_COLOR = (255,0,0) # red color
TEXT_COLOR = (255,255,255) # white color

In [34]:
class MyCustomDataset(Dataset):
    def __init__(self, json_path, transform = None):        
        self.json_file = json_path
        self.trnasform = transform

    def __getitem__(self, index): 
        json_path = self.json_file[index]
        # open json file
        with open(json_path) as f:
            json_file = json.loads(f.read()) ## json 라이브러리 이용

        category_ids, category_id_list= [], []
        for item in json_file["categories"]:
            category_ids.append(item["id"])
            category_id_list.append(item["name"])

        # category dictionary
        category_id_to_name = dict(zip(category_ids, category_id_list))      
        image_path= os.path.join("./2022.12/12.16_d53_data/data", 
                                json_file['images'][0]['file_name'])
        image = cv2.imread(image_path)
             
        bboxes = []
        annotations = sorted(json_file["annotations"], key= itemgetter("category_id"))
        for i in range(len(annotations)):
            bboxes.append(annotations[i]["bbox"])
        return image, bboxes, category_ids, category_id_to_name

    def __len__(self):
        return len(self.json_file)

In [35]:
### visualize a single bounding box on the image
def visualize_bbox(image, bboxes, category_ids, category_id_to_name, color=BOX_COLOR, thickness=2) :
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids) :
        class_name = category_id_to_name[category_id]
        # print('class_name >>> ', class_name)
        
        x_min, y_min, w, h = bbox
        x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)

        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
        cv2.putText(img, text=class_name, org=(x_min, y_min+30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=color, thickness=thickness)
    cv2.imshow("test", img)
    cv2.waitKey(0)
    


In [40]:
json_path = "./instances_default.json"
image = cv2.imread("./01.jpg")

In [37]:
# 객체를 선언한 다음에 변수에 담아야 하고 이를 통해 호출한다. self가 들어갔으면 멤버 변수
data = MyCustomDataset(json_path)

In [41]:
for item in data:
    image               = item[0]
    bboxes              = item[1]
    category_ids        = item[2]
    category_id_to_name = item[3]

PermissionError: [Errno 13] Permission denied: '.'

In [ ]:
# transforms
transfor = A.Compose([
    # A.RandomSizedBBoxSafeCrop(width=448, height=336, erosion_rate=0.2),
    A.VerticalFlip(p=1),
    A.HorizontalFlip(p=1),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))

In [ ]:
transformed = transfor(image= image, bboxes= bboxes, category_ids= category_ids)

visualize_bbox(transformed['image'], transformed['bboxes'], transformed['category_ids'],
               category_id_to_name,
               color= BOX_COLOR, thickness=2)

TypeError: image must be numpy array type